In [2]:
import sqlalchemy as sa
from sqlalchemy import orm
from dotenv import load_dotenv
import pandas as pd
import os
import sys

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    

load_dotenv()

conn_string = 'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/cosmetic'.format(
    db_user=os.getenv('DB_USER'),
    db_password=os.getenv('DB_PASSWORD'),
    db_host=os.getenv('DB_HOST'),
    db_port=os.getenv('DB_PORT'),
)
engine = sa.create_engine(conn_string)

In [2]:
# memorize products' release date and max price
with engine.connect() as conn:
    product_df = pd.read_sql('''
            SELECT 
                e.product_id,
                e.category_id,
                MIN(e.event_time)::DATE as release_date,
                MAX(e.price) as max_price
            FROM 
                events e
            GROUP BY
                e.product_id, e.category_id
            ORDER BY
                e.category_id ASC,
                e.product_id ASC
        ''',
        con=conn
    )

    conn.execute(sa.text('TRUNCATE TABLE products'))

    product_df.to_sql(
        name='products', 
        con=conn, 
        if_exists='append', 
        index=False,
        chunksize=1000,
        dtype={
            'release_date': sa.types.DATE
        }
    )

    conn.commit()

print('Count:', product_df.shape[0])
product_df.head(15)

Count: 56190


,product_id,category_id,release_date,max_price
0,5916194,1487580004807082827,2019-12-25,11.70
1,5916195,1487580004807082827,2019-12-25,12.41
2,5706112,1487580004832248652,2019-10-09,14.13
3,5706113,1487580004832248652,2019-10-01,15.08
4,5706114,1487580004832248652,2019-10-01,14.13
5,5706136,1487580004832248652,2019-10-02,14.13
6,5706137,1487580004832248652,2019-10-01,15.08
7,5706139,1487580004832248652,2019-10-01,14.13
8,5706141,1487580004832248652,2019-10-01,14.13
9,5706301,1487580004832248652,2019-10-31,14.13


In [6]:
# ranking by sales of the previous month
from collections import defaultdict
import datetime

from schemas.data_by_week import DataByWeek
from schemas.product import Product

with orm.Session(engine) as session, session.begin():
    d = datetime.date(2019, 9, 30)
    end = datetime.date(2020, 3, 1)

    products = dict[tuple[int, int], Product]()
    sales = defaultdict[tuple[int, int], tuple[int]](int)
    memorized = defaultdict[datetime.date, defaultdict[tuple[int, int], int]](lambda: defaultdict(int))

    for row in session.scalars(sa.select(Product)).all():
        products[(row.product_id, row.category_id)] = row

    def key_func(data: DataByWeek):
        key = (data.product_id, data.category_id)
        product = products[key]
        return -sales[key], data.date - product.release_date, product.max_price
    
    while d < end:
        rows = list[DataByWeek]()
        categories = defaultdict[int, list[DataByWeek]](list[DataByWeek])
        for row in session.scalars(sa.select(DataByWeek).where(DataByWeek.date == d)):
            rows.append(row)
            categories[row.category_id].append(row)

        for category in categories.values():
            category.sort(key=key_func)
            for index, row in enumerate(category):
                row.rank_in_category = index + 1

        rows.sort(key=key_func)
        for index, row in enumerate(rows):
            row.rank = index + 1

        pre_d = d - datetime.timedelta(weeks=4)
        for row in rows:
            key = (row.product_id, row.category_id)
            sales[key] += row.purchase - memorized[pre_d][key]
            memorized[d][key] = row.purchase
        del memorized[pre_d]

        session.flush()
        d += datetime.timedelta(weeks=1)
    session.commit()

In [8]:
with engine.connect() as conn:
    df = pd.read_sql('''
        SELECT
            AVG(d.rank) as mean_rank,
            STDDEV(d.rank) AS std_rank,
            AVG(d.rank_in_category) as mean_rank_in_category,
            STDDEV(d.rank_in_category) AS std_rank_in_category,
            AVG(d.max_price) as mean_price,
            STDDEV(d.max_price) AS std_price,
            AVG(d.view) as mean_view,
            STDDEV(d.view) AS std_view,
            AVG(d.cart) as mean_cart,
            STDDEV(d.cart) AS std_cart,
            AVG(d.remove_from_cart) as mean_remove_from_cart,
            STDDEV(d.remove_from_cart) AS std_remove_from_cart,
            AVG(d.purchase) as mean_purchase,
            STDDEV(d.purchase) AS std_purchase,
            AVG(d.date - p.release_date) as mean_days_on_shelf,
            STDDEV(d.date - p.release_date) AS std_days_on_shelf
        FROM
            data_by_week d
            INNER JOIN products p ON p.product_id = d.product_id AND p.category_id = d.category_id
    ''', con=conn)

    conn.execute(sa.text('TRUNCATE TABLE parameters'))

    for col in df.columns.to_list():
        conn.execute(sa.text(f"INSERT INTO parameters(name, value) VALUES ('{col}', {df[col][0]})"))
    
    conn.commit()